# Share Tensor
### With Duet Clients

## Import libs

In [1]:
import syft as sy
import torch

from sympc.session import Session
from sympc.session import SessionManager

from sympc.tensor import MPCTensor

sy.load("sympc")
sy.logger.add(sink="./example.log")

## Create the clients

In [6]:
alice = sy.VirtualMachine(name="alice")
bob = sy.VirtualMachine(name="bob")

alice_client = alice.get_client()
bob_client = bob.get_client()

## Create a session

The session is used to send some config information only once between the parties.
This information can be:
* the ring size in which we do the computation
* the precision and base
* the approximation methods we are using for different functions (TODO)

In [7]:
session = Session(parties=[alice_client, bob_client])
print(session)

## Send the session to all the parties

In [8]:
SessionManager.setup_mpc(session)

In [9]:
x = MPCTensor(secret=torch.tensor([1,2,3,4]), session=session)

In [10]:
print(x)

[MPCTensor]
	| <VirtualMachineClient: alice Client> -> ShareTensorPointer
	| <VirtualMachineClient: bob Client> -> ShareTensorPointer


In [11]:
# The values does not make sense for any user, unless they "reconstruct" the original tensor
for val in x.share_ptrs:
    print(val.get()) ## Attention! get() destroys the pointer

[ShareTensor]
	| [FixedPointEncoder]: precision: 16, base: 2
	| Data: tensor([4497613577609696743, 2913873267072586058, 5176484048171862467,
         391990069951415545])
[ShareTensor]
	| [FixedPointEncoder]: precision: 16, base: 2
	| Data: tensor([-4497613577609631207, -2913873267072454986, -5176484048171665859,
         -391990069951153401])


In [12]:
x_secret = torch.tensor([1,2,3,4])
x = MPCTensor(secret=x_secret, session=session)

In [13]:
print(x.reconstruct()) ## Attention! Also reconstruct, destroys the pointers

tensor([1., 2., 3., 4.])


## Secret is shared by the orchestrator
- the orchestrator generates shares locally
- distributes them to the parties

### Public Operations

In [14]:
x_secret = torch.tensor([[1,2],[3,4]])
x = MPCTensor(secret=x_secret, session=session)

y = torch.tensor([[5,6],[7,8]])

In [15]:
print("[Priv + Pub] X + Y =\n", (x + y).reconstruct())
print("[Pub + Pub] X + Y =\n", x_secret + y)

[Priv + Pub] X + Y =
 tensor([[ 6.,  8.],
        [10., 12.]])
[Pub + Pub] X + Y =
 tensor([[ 6,  8],
        [10, 12]])


In [16]:
print("[Priv - Pub] X - Y =\n", (x - y).reconstruct())
print("[Pub - Pub] X - Y =\n", x_secret - y)

[Priv - Pub] X - Y =
 tensor([[-4., -4.],
        [-4., -4.]])
[Pub - Pub] X - Y =
 tensor([[-4, -4],
        [-4, -4]])


In [17]:
print("[Priv * Pub] X * Y =\n", (x * y).reconstruct())
print("[Pub * Pub] X * Y =\n", x_secret * y)

[Priv * Pub] X * Y =
 tensor([[ 5., 12.],
        [21., 32.]])
[Pub * Pub] X * Y =
 tensor([[ 5, 12],
        [21, 32]])


In [18]:
print("[Priv @ Pub] X * Y =\n", (x @ y).reconstruct())
print("[Pub @ Pub] X * Y =\n", x_secret @ y)

[Priv @ Pub] X * Y = tensor([[19., 22.],
        [43., 50.]])
[Pub @ Pub] X * Y = tensor([[19, 22],
        [43, 50]])


### Private Operations

In [21]:
x_secret = torch.tensor([[1,2],[3,4]])
x = MPCTensor(secret=x_secret,session=session)

y_secret = torch.tensor([[5,6],[7,8]])
y = MPCTensor(secret=y_secret, session=session)

In [25]:
print("[Priv + Priv] X + Y =\n", (x + y).reconstruct())
print("[Pub + Pub] X + Y =\n", x_secret + y_secret)

[Priv + Priv] X + Y =
 tensor([[ 6.,  8.],
        [10., 12.]])
[Pub + Pub] X + Y =
 tensor([[ 6,  8],
        [10, 12]])


In [26]:
print("[Priv - Priv] X - Y =\n", (x - y).reconstruct())
print("[Pub - Pub] X - Y =\n", x_secret - y_secret)

[Priv - Priv] X - Y =
 tensor([[-4., -4.],
        [-4., -4.]])
[Pub - Pub] X - Y =
 tensor([[-4, -4],
        [-4, -4]])


In [27]:
print("[Priv * Priv] X * Y =\n", (x * y).reconstruct())
print("[Pub * Pub] X * Y =\n", x_secret * y_secret)

[Priv * Priv] X * Y =
 tensor([[ 5., 12.],
        [21., 32.]])
[Pub * Pub] X * Y =
 tensor([[ 5, 12],
        [21, 32]])


In [28]:
print("[Priv @ Pub] X * Y =\n", (x @ y).reconstruct())
print("[Pub @ Pub] X * Y =\n", x_secret @ y_secret)

[Priv @ Pub] X * Y =
 tensor([[19., 22.],
        [43., 50.]])
[Pub @ Pub] X * Y =
 tensor([[19, 22],
        [43, 50]])


## Private Secret - owned by one party
- are used more generators to construct PRZS (Pseudo-Random-Zero-Shares)
- the party that has the secret would add it to their own share

### Public Operations

In [44]:
x_secret = alice_client.torch.Tensor([[1,2],[3,4]]) # at Alice
x = MPCTensor(secret=x_secret, shape=(2, 2), session=session)

y = torch.tensor([[5,6],[7,8]], dtype=torch.float)

In [37]:
print("[Priv + Pub] X + Y =\n", (x + y).reconstruct())
print("[Pub + Pub] X + Y =\n", (x_secret + y).get()) # On Alice side

[Priv + Pub] X + Y =
 tensor([[ 6.,  8.],
        [10., 12.]])
[Pub + Pub] X + Y =
 tensor([[ 6.,  8.],
        [10., 12.]])


In [38]:
print("[Priv - Pub] X - Y =\n", (x - y).reconstruct())
print("[Pub - Pub] X - Y =\n", (x_secret - y).get()) # On Alice side

[Priv - Pub] X - Y =
 tensor([[-4., -4.],
        [-4., -4.]])
[Pub - Pub] X - Y =
 tensor([[-4., -4.],
        [-4., -4.]])


In [39]:
print("[Priv * Pub] X * Y =\n", (x * y).reconstruct())
print("[Pub * Pub] X * Y =\n", (x_secret * y).get()) # On Alice side

[Priv * Pub] X * Y =
 tensor([[ 5., 12.],
        [21., 32.]])
[Pub * Pub] X * Y =
 tensor([[ 5., 12.],
        [21., 32.]])


In [46]:
print("[Priv @ Pub] X * Y =\n", (x @ y).reconstruct())
print("[Pub @ Pub] X * Y =\n", (x_secret @ y).get()) # On Alice side

[Priv @ Pub] X * Y =
 tensor([[19., 22.],
        [43., 50.]])
[Pub @ Pub] X * Y =
 tensor([[19., 22.],
        [43., 50.]])


### Private Operations

In [57]:
x_secret = alice_client.torch.Tensor([[1,2],[3,4]]) # at Alice
x = MPCTensor(secret=x_secret, shape=(2, 2), session=session)

y_secret = bob_client.torch.Tensor([[5,6],[7,8]]) # at Bob
y = MPCTensor(secret=y_secret, shape=(2, 2), session=session)

In [58]:
print("[Priv + Priv] X + Y =\n", (x + y).reconstruct())

# We can not simply add them because they are at different locations
# That is why we need to first get them before adding them
print("[Pub + Pub] X + Y =\n", x_secret.get_copy() + y_secret.get_copy())

[Priv + Priv] X + Y =
 tensor([[ 6.,  8.],
        [10., 12.]])
[Pub + Pub] X + Y =
 tensor([[ 6.,  8.],
        [10., 12.]])


In [59]:
print("[Priv - Priv] X - Y =\n", (x - y).reconstruct())
print("[Pub - Pub] X - Y =\n", x_secret.get_copy() - y_secret.get_copy())

[Priv - Priv] X - Y =
 tensor([[-4., -4.],
        [-4., -4.]])
[Pub - Pub] X - Y =
 tensor([[-4., -4.],
        [-4., -4.]])


In [60]:
print("[Priv * Priv] X * Y =\n", (x * y).reconstruct())
print("[Pub * Pub] X * Y =\n", x_secret.get_copy() * y_secret.get_copy())

[Priv * Priv] X * Y =
 tensor([[ 5., 12.],
        [21., 32.]])
[Pub * Pub] X * Y =
 tensor([[ 5., 12.],
        [21., 32.]])


In [62]:
print("[Priv @ Pub] X * Y =\n", (x @ y).reconstruct())
print("[Pub @ Pub] X * Y =\n", x_secret.get_copy() @ y_secret.get_copy())

[Priv @ Pub] X * Y =
 tensor([[19., 22.],
        [43., 50.]])
[Pub @ Pub] X * Y =
 tensor([[19., 22.],
        [43., 50.]])
